## Hyper parameter
evaluating the performance model<br>
Load all the libraries needed

In [2]:
import sys
import numpy as np
import os
import os.path as op
import re
import pandas as pd
import imageio
from skimage.io import imread, imsave
from skimage.transform import resize
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, auc
from scipy import interp


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import datetime
from PIL import Image
from itertools import chain
import csv
import random
import pickle

import warnings
warnings.filterwarnings('ignore');
plt.style.use('ggplot')
%matplotlib inline

In [5]:
def loadDataAndLabels(myDataDir):
    imgFiles = []
    myLabel = []
    with open(myDataDir+'/aa-labels.csv','r') as fi:
        reader =  csv.reader(fi)
        ignoreHead = 0
        for row in reader:
            #print(row)
            if len(row) > 0:
                if ignoreHead == 0:
                    ignoreHead = 1
                else:
                    imgFiles.append(row[0])
                    myLabel.append(row[1])
    fi.close()
    num_ims = len(imgFiles)
    num_rows = 100
    num_cols = 100
    num_chans = 3
    resize_factor = 4
    num_rows_resize = int(num_rows/resize_factor)
    num_cols_resize = int(num_cols/resize_factor)
    myDataNp = np.zeros((num_ims,num_rows_resize,num_cols_resize,num_chans))
    myLabelNp = np.array(myLabel)
    i = 0
    for imgName in imgFiles:
        img_data = resize(imread(myDataDir+"/"+imgName+".jpg"),(num_rows_resize,num_cols_resize),mode='constant')
        myDataNp[i,:,:,:] = img_data[:,:,:]
        if i % 1000 == 0:
            print(i,": loaded img - ", myDataDir,"/",imgName,".jpg")
        i = i+1
    myDataNp = myDataNp.reshape(num_ims,num_rows_resize*num_cols_resize*num_chans)
    return(myDataNp,myLabelNp)

In [6]:
TR_DATA_DIR = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1'
# Load the training set
(trData,trLabel) = loadDataAndLabels(TR_DATA_DIR)

0 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Braeburn_0_100_f0 .jpg
1000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Golden 2_107_100_f7 .jpg
2000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Granny Smith_113_100_f1 .jpg
3000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Red 2_117_100_f3 .jpg
4000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Red Delicious_125_100_f2 .jpg
5000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Red Yellow 2_156_100_f7 .jpg
6000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Avocado_136_100_f2 .jpg
7000 : loaded img -  C:/Users/sonali/Deskto

In [7]:
TE_DATA_DIR = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1'
# Load the test set
(teData,teLabel) = loadDataAndLabels(TE_DATA_DIR)

0 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Apple Braeburn_321_100_f0 .jpg
1000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Apple Red Yellow 2_112_100_f4 .jpg
2000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Carambula_146_100_f8 .jpg
3000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Granadilla_105_100_f0 .jpg
4000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Huckleberry_100_100_f11 .jpg
5000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Mangostan_132_100_f6 .jpg
6000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Pear_322_100_f1 .jpg
7000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_med

In [8]:
#X_train=trData,y_train=trLabel
#X_test=teData,y_test=teLabel

for the optimal perforamce we need to have input feature of same value so standardScaler, reduce the dimension for the
better perforamce by PCA, logistic regression for the  linear classifeir using pipeline

In [9]:
pipe_logreg = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(penalty='l2', 
                                           random_state=1, 
                                           C=100.0))

pipe_logreg.fit(trData, trLabel)
y_pred = pipe_logreg.predict(teData)
print('Test Accuracy: {}' .format(pipe_logreg.score(teData, teLabel)))

Test Accuracy: 0.4240583232077764


### Kfold cross validation
training dataset is split in to 20 folds from that 19 folds are used in the model training and 1 is used to evaluate 
the preformace of the model

In [10]:
kfold = StratifiedKFold(n_splits=20,random_state=1).split(trData, trLabel)

scores = []
for k, (train, test) in enumerate(kfold):
    pipe_logreg.fit(trData[train], trLabel[train])
    score = pipe_logreg.score(trData[test], trLabel[test])
    scores.append(score)
    print('Fold:{}, Accuracy:{}'.format(k+1, score))


Fold:1, Accuracy:0.4238116392714349
Fold:2, Accuracy:0.4260328742780986
Fold:3, Accuracy:0.4362505553087517
Fold:4, Accuracy:0.4255886272767659
Fold:5, Accuracy:0.4264771212794314
Fold:6, Accuracy:0.4264771212794314
Fold:7, Accuracy:0.43066666666666664
Fold:8, Accuracy:0.42444444444444446
Fold:9, Accuracy:0.41955555555555557
Fold:10, Accuracy:0.4235555555555556
Fold:11, Accuracy:0.43422222222222223
Fold:12, Accuracy:0.4191111111111111
Fold:13, Accuracy:0.4182222222222222
Fold:14, Accuracy:0.4117385504668742
Fold:15, Accuracy:0.41218319253001334
Fold:16, Accuracy:0.4295242329924411
Fold:17, Accuracy:0.4308581591818586
Fold:18, Accuracy:0.4359430604982206
Fold:19, Accuracy:0.42857142857142855
Fold:20, Accuracy:0.43168669336893634


average accuray and std devation

In [11]:
print('Cross validation accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

Cross validation accuracy: 0.426 +/- 0.007


In [12]:
scores = cross_val_score(estimator=pipe_logreg,X=trData, y=trLabel,cv=20, n_jobs=1)
print('cross validation accuracy scores:\n{}'.format(scores))

cross validation accuracy scores:
[0.42381164 0.42603287 0.43625056 0.42558863 0.42647712 0.42647712
 0.43066667 0.42444444 0.41955556 0.42355556 0.43422222 0.41911111
 0.41822222 0.41173855 0.41218319 0.42952423 0.43085816 0.43594306
 0.42857143 0.43168669]


In [13]:
print('cross validation accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

cross validation accuracy: 0.426 +/- 0.007
